In [1]:
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

import pandas as pd
import numpy as np

from pathlib import Path
from datetime import datetime
import multiprocessing
import logging

cores = multiprocessing.cpu_count()
print(f'cores: {cores}')

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

cores: 4


In [2]:
path_data = Path('../../data/corpora/clean_tutby_126784.csv')
# path_data = Path('../../data/corpora/clean_tutby_126784_header.csv')

# path_d2v = Path('../../data/model/d2v/model_d2v_clean_tutby_126784.model')
path_d2v = Path('../../data/model/d2v/model_d2v_v2_clean_tutby_126784.model')
# path_d2v = Path('../../data/model/d2v/model_d2v_clean_tutby_126784_header.model')

# path_embedding = Path('../../data/emb/emb_clean_tutby_126784_d2v.npy')
path_embedding = Path('../../data/emb/emb_clean_tutby_126784_d2v_v2.npy')
# path_embedding = Path('../../data/emb/emb_clean_tutby_126784_header_d2v.npy')

column_name = 'document'
# column_name = 'header'

In [3]:
data = pd.read_csv(path_data)

documents = data[column_name]
documents = documents.fillna('')

print(data.shape)
data.head(5)

(126784, 6)


,url,label,header,date,document,tags
0,https://news.tut.by/550306.html,Футбол,"Тренер ""Шахтера"": Оправдываться не хочу. Все в...",2017-07-06T21:35:00+03:00,главный тренер солигорский шахтер олег кубарев...,['футбол']
1,https://news.tut.by/550307.html,Общество,"""Зацветет"" ли каменная роза на ул. Комсомольск...",2017-07-07T09:25:00+03:00,план восстановление рисунок пока художник илья...,"['архитектура', 'живопись', 'ЖКХ']"
2,https://news.tut.by/550308.html,Общество,Фотофакт. Скамейка в виде пожарной машины появ...,2017-07-07T09:27:00+03:00,областной управление мчс день пожарный служба ...,['министерства']
3,https://news.tut.by/550309.html,Футбол,Станислав Драгун дебютировал за БАТЭ в матче с...,2017-07-06T22:11:00+03:00,чемпион беларусь бате воспользоваться пауза че...,"['футбол', 'БАТЭ']"
4,https://news.tut.by/550310.html,В мире,Генпрокурор Украины пообещал открыть уголовное...,2017-07-06T22:28:00+03:00,генпрокуратура украина открывать уголовный про...,"['Ситуация в Украине', 'государственные перево..."


In [4]:
%%time
print(datetime.now())

# params_doc2vec = {  # v1
#     'dm': 0,
#     'vector_size': 300,
#     'window': 15,
#     'min_count': 20,
#     'sample': 1e-5,
#     'workers': cores,
#     'epochs': 40,
#     'hs': 0,
#     'negative': 5,
#     'dbow_words': 1,
# }

params_doc2vec = {  # v2
    'dm': 0,
    'vector_size': 300,
    'window': 15,
    'min_count': 50,
    'sample': 1e-5,
    'workers': cores,
    'epochs': 40,
    'hs': 1,
    'negative': 0,
    'dbow_words': 1,
}

documents_gensim = [TaggedDocument(document.split(), [index]) for index, document in documents.items()]

model = Doc2Vec(documents=documents_gensim, **params_doc2vec)

model.save(str(path_d2v))
model = Doc2Vec.load(str(path_d2v))

embeddings = model.docvecs.vectors_docs

with open(path_embedding, 'wb') as file:
    np.save(file, embeddings)

with open(path_embedding, 'rb') as file:
    embeddings = np.load(file)
    
print(embeddings.shape)

2020-11-29 12:56:22.746777


2020-11-29 12:56:28,971 : INFO : collecting all words and their counts
2020-11-29 12:56:28,971 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-11-29 12:56:29,616 : INFO : PROGRESS: at example #10000, processed 2466256 words (3835565/s), 99885 word types, 10000 tags
2020-11-29 12:56:30,346 : INFO : PROGRESS: at example #20000, processed 5035139 words (3511244/s), 143097 word types, 20000 tags
2020-11-29 12:56:31,079 : INFO : PROGRESS: at example #30000, processed 7623391 words (3545603/s), 176652 word types, 30000 tags
2020-11-29 12:56:31,776 : INFO : PROGRESS: at example #40000, processed 10215111 words (3706362/s), 205336 word types, 40000 tags
2020-11-29 12:56:32,450 : INFO : PROGRESS: at example #50000, processed 12787803 words (3825163/s), 231745 word types, 50000 tags
2020-11-29 12:56:33,122 : INFO : PROGRESS: at example #60000, processed 15423913 words (3937739/s), 256913 word types, 60000 tags
2020-11-29 12:56:33,783 : INFO : PROGRESS: at exa

2020-11-29 12:58:03,131 : INFO : EPOCH 1 - PROGRESS: at 24.58% examples, 56582 words/s, in_qsize 7, out_qsize 0
2020-11-29 12:58:04,261 : INFO : EPOCH 1 - PROGRESS: at 25.04% examples, 56458 words/s, in_qsize 7, out_qsize 0
2020-11-29 12:58:05,378 : INFO : EPOCH 1 - PROGRESS: at 25.61% examples, 56581 words/s, in_qsize 7, out_qsize 0
2020-11-29 12:58:06,410 : INFO : EPOCH 1 - PROGRESS: at 25.99% examples, 56391 words/s, in_qsize 8, out_qsize 0
2020-11-29 12:58:07,429 : INFO : EPOCH 1 - PROGRESS: at 26.41% examples, 56295 words/s, in_qsize 7, out_qsize 0
2020-11-29 12:58:08,498 : INFO : EPOCH 1 - PROGRESS: at 26.88% examples, 56266 words/s, in_qsize 8, out_qsize 0
2020-11-29 12:58:09,546 : INFO : EPOCH 1 - PROGRESS: at 27.32% examples, 56258 words/s, in_qsize 8, out_qsize 0
2020-11-29 12:58:10,591 : INFO : EPOCH 1 - PROGRESS: at 27.74% examples, 56248 words/s, in_qsize 7, out_qsize 0
2020-11-29 12:58:11,646 : INFO : EPOCH 1 - PROGRESS: at 28.22% examples, 56225 words/s, in_qsize 7, out_

2020-11-29 12:59:20,740 : INFO : EPOCH 1 - PROGRESS: at 59.46% examples, 55872 words/s, in_qsize 7, out_qsize 0
2020-11-29 12:59:21,798 : INFO : EPOCH 1 - PROGRESS: at 59.95% examples, 55889 words/s, in_qsize 7, out_qsize 0
2020-11-29 12:59:22,801 : INFO : EPOCH 1 - PROGRESS: at 60.39% examples, 55902 words/s, in_qsize 7, out_qsize 0
2020-11-29 12:59:23,842 : INFO : EPOCH 1 - PROGRESS: at 60.89% examples, 55895 words/s, in_qsize 8, out_qsize 0
2020-11-29 12:59:24,916 : INFO : EPOCH 1 - PROGRESS: at 61.34% examples, 55906 words/s, in_qsize 7, out_qsize 0
2020-11-29 12:59:25,937 : INFO : EPOCH 1 - PROGRESS: at 61.78% examples, 55897 words/s, in_qsize 7, out_qsize 0
2020-11-29 12:59:26,981 : INFO : EPOCH 1 - PROGRESS: at 62.23% examples, 55867 words/s, in_qsize 7, out_qsize 0
2020-11-29 12:59:27,998 : INFO : EPOCH 1 - PROGRESS: at 62.66% examples, 55860 words/s, in_qsize 7, out_qsize 0
2020-11-29 12:59:29,021 : INFO : EPOCH 1 - PROGRESS: at 63.17% examples, 55857 words/s, in_qsize 7, out_

2020-11-29 13:00:38,066 : INFO : EPOCH 1 - PROGRESS: at 94.59% examples, 56228 words/s, in_qsize 7, out_qsize 0
2020-11-29 13:00:39,113 : INFO : EPOCH 1 - PROGRESS: at 95.07% examples, 56232 words/s, in_qsize 8, out_qsize 0
2020-11-29 13:00:40,161 : INFO : EPOCH 1 - PROGRESS: at 95.53% examples, 56253 words/s, in_qsize 7, out_qsize 0
2020-11-29 13:00:41,166 : INFO : EPOCH 1 - PROGRESS: at 96.03% examples, 56267 words/s, in_qsize 8, out_qsize 0
2020-11-29 13:00:42,256 : INFO : EPOCH 1 - PROGRESS: at 96.45% examples, 56249 words/s, in_qsize 8, out_qsize 0
2020-11-29 13:00:43,336 : INFO : EPOCH 1 - PROGRESS: at 96.87% examples, 56221 words/s, in_qsize 7, out_qsize 0
2020-11-29 13:00:44,381 : INFO : EPOCH 1 - PROGRESS: at 97.41% examples, 56233 words/s, in_qsize 7, out_qsize 0
2020-11-29 13:00:45,446 : INFO : EPOCH 1 - PROGRESS: at 97.90% examples, 56238 words/s, in_qsize 7, out_qsize 0
2020-11-29 13:00:46,463 : INFO : EPOCH 1 - PROGRESS: at 98.40% examples, 56257 words/s, in_qsize 7, out_

...

2020-11-29 15:18:57,455 : INFO : EPOCH 40 - PROGRESS: at 0.31% examples, 27125 words/s, in_qsize 8, out_qsize 0
2020-11-29 15:18:58,635 : INFO : EPOCH 40 - PROGRESS: at 0.67% examples, 30779 words/s, in_qsize 8, out_qsize 0
2020-11-29 15:18:59,720 : INFO : EPOCH 40 - PROGRESS: at 1.06% examples, 33739 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:19:00,752 : INFO : EPOCH 40 - PROGRESS: at 1.37% examples, 34031 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:19:01,760 : INFO : EPOCH 40 - PROGRESS: at 1.81% examples, 37024 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:19:02,789 : INFO : EPOCH 40 - PROGRESS: at 2.25% examples, 38882 words/s, in_qsize 8, out_qsize 0
2020-11-29 15:19:03,818 : INFO : EPOCH 40 - PROGRESS: at 2.69% examples, 40202 words/s, in_qsize 8, out_qsize 0
2020-11-29 15:19:04,925 : INFO : EPOCH 40 - PROGRESS: at 3.12% examples, 40961 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:19:05,967 : INFO : EPOCH 40 - PROGRESS: at 3.51% examples, 41772 words/s, in_qsize 7, out_

2020-11-29 15:20:14,150 : INFO : EPOCH 40 - PROGRESS: at 32.62% examples, 50518 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:20:15,150 : INFO : EPOCH 40 - PROGRESS: at 33.10% examples, 50617 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:20:16,210 : INFO : EPOCH 40 - PROGRESS: at 33.65% examples, 50772 words/s, in_qsize 8, out_qsize 0
2020-11-29 15:20:17,210 : INFO : EPOCH 40 - PROGRESS: at 34.14% examples, 50863 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:20:18,235 : INFO : EPOCH 40 - PROGRESS: at 34.65% examples, 50983 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:20:19,275 : INFO : EPOCH 40 - PROGRESS: at 35.19% examples, 51085 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:20:20,300 : INFO : EPOCH 40 - PROGRESS: at 35.72% examples, 51200 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:20:21,356 : INFO : EPOCH 40 - PROGRESS: at 36.21% examples, 51287 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:20:22,366 : INFO : EPOCH 40 - PROGRESS: at 36.67% examples, 51359 words/s, in_qsiz

2020-11-29 15:21:30,645 : INFO : EPOCH 40 - PROGRESS: at 66.38% examples, 52327 words/s, in_qsize 8, out_qsize 0
2020-11-29 15:21:31,658 : INFO : EPOCH 40 - PROGRESS: at 66.89% examples, 52399 words/s, in_qsize 8, out_qsize 0
2020-11-29 15:21:32,779 : INFO : EPOCH 40 - PROGRESS: at 67.38% examples, 52387 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:21:33,886 : INFO : EPOCH 40 - PROGRESS: at 67.73% examples, 52290 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:21:34,984 : INFO : EPOCH 40 - PROGRESS: at 68.04% examples, 52178 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:21:36,030 : INFO : EPOCH 40 - PROGRESS: at 68.38% examples, 52104 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:21:37,187 : INFO : EPOCH 40 - PROGRESS: at 68.83% examples, 52086 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:21:38,267 : INFO : EPOCH 40 - PROGRESS: at 69.35% examples, 52137 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:21:39,330 : INFO : EPOCH 40 - PROGRESS: at 69.81% examples, 52154 words/s, in_qsiz

2020-11-29 15:22:47,189 : INFO : EPOCH 40 - PROGRESS: at 97.73% examples, 52153 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:22:48,382 : INFO : EPOCH 40 - PROGRESS: at 97.92% examples, 51988 words/s, in_qsize 8, out_qsize 0
2020-11-29 15:22:49,385 : INFO : EPOCH 40 - PROGRESS: at 98.26% examples, 51952 words/s, in_qsize 8, out_qsize 0
2020-11-29 15:22:50,619 : INFO : EPOCH 40 - PROGRESS: at 98.63% examples, 51855 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:22:51,636 : INFO : EPOCH 40 - PROGRESS: at 98.86% examples, 51761 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:22:52,663 : INFO : EPOCH 40 - PROGRESS: at 99.15% examples, 51668 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:22:53,815 : INFO : EPOCH 40 - PROGRESS: at 99.35% examples, 51523 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:22:54,965 : INFO : EPOCH 40 - PROGRESS: at 99.71% examples, 51454 words/s, in_qsize 7, out_qsize 0
2020-11-29 15:22:55,634 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-

(126784, 300)
Wall time: 2h 26min 38s
